In [1]:
# Copyright (C) 2023 Antonio Rodriguez
# 
# This file is part of T1DM_WARIFA.
# 
# T1DM_WARIFA is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
# 
# T1DM_WARIFA is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
# 
# You should have received a copy of the GNU General Public License
# along with T1DM_WARIFA.  If not, see <http://www.gnu.org/licenses/>.

import os 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import pickle
from typing import Dict

import sys 
sys.path.append("..")

from utils import *

from sensor_params import *

from models.multi_step.naive_model import *
from models.multi_step.naive_model import naive_model as get_naive_multi_step

from evaluation.multi_step.evaluation import model_evaluation as multi_step_model_evaluation
from evaluation.multi_step.evaluation import model_evaluation_close_loop as multi_step_model_evaluation_refeed

 # Dictionaty with the configurations to train all models 
from training_configs import * 

# Dataset path 
DATABASE_PATH = r"C:\Users\aralmeida\OneDrive - Universidad de Las Palmas de Gran Canaria\Doctorado\Bases de datos\WARIFA\Mikael T1DM"

filename = "MIKAEL_data.json"

parent_directory = r"C:\Users\aralmeida\Downloads"
experiments_folder = r"\T1DM_pred_experiments"

with open(DATABASE_PATH+'\CGM.pk1', 'rb') as f:
    sgv_data_dict = pickle.load(f)
os.chdir(parent_directory)


In [3]:
X, Y, X_times, Y_times = get_CGM_X_Y_multistep(sgv_data_dict, sensor, N, step, PH, experiments_folder, plot=False, verbose = 0)  

X_norm = (X - np.min(X))/(np.max(X) - np.min(X))
Y_norm = (Y - np.min(X))/(np.max(X) - np.min(X))

X_train = X_norm[np.where(X_times[:,N-1] <= pd.to_datetime('2021-05-31 00:00:00'))[0]]
Y_train = Y_norm[np.where((Y_times[:,0] < pd.to_datetime('2021-05-30 23:59:59')))[0]]
X_test = X_norm[np.where((X_times[:,N-1] > pd.to_datetime('2021-06-01 00:00:00')))[0]]
Y_test = Y_norm[np.where((Y_times[:,0] > pd.to_datetime('2021-06-01 00:00:00')))[0]] # Left non-normalized to compute the metrics

predicted_points = round(PH/sensor['SAMPLE_PERIOD'])

# Declare an SVM regressor
from sklearn.svm import SVR

# Create the model
model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)

# Train the model
model.fit(X_train, Y_train)

# Predict the test set
Y_pred = model.predict(X_test)







# Non-refeed evaluation
results_normal_eval = multi_step_model_evaluation(N, PH, model_name, normalization, X_test, Y_test, predicted_points, X, loss_function)
os.chdir('..')



Number of blocks is 335



ValueError: y should be a 1d array, got an array of shape (120377, 12) instead.